In [22]:
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Okt

import pandas as pd
from datetime import datetime
import time
import re

In [2]:
# 정치 https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100
# 경제 https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101
# 사회 https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102
# 생활문화 https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103
# IT과학 https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105
# 세계 https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=104

In [6]:
# 링크 확인

current_call = 1
last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

politics_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100'
politics_web = requests.get(politics_url).content
politics_source = BeautifulSoup(politics_web, 'html.parser')

politics_urls_list = []

for politics_urls in politics_source.find_all('a', {'class' : "sh_text_headline nclicks(cls_pol.clsart)"}):
    if politics_urls["href"].startswith("https://n.news.naver.com"):
        politics_urls_list.append(politics_urls["href"])

politics_urls_list


['https://n.news.naver.com/mnews/article/001/0014469515?sid=100',
 'https://n.news.naver.com/mnews/article/001/0014470138?sid=100',
 'https://n.news.naver.com/mnews/article/214/0001326896?sid=100',
 'https://n.news.naver.com/mnews/article/001/0014469974?sid=100',
 'https://n.news.naver.com/mnews/article/016/0002257706?sid=100',
 'https://n.news.naver.com/mnews/article/028/0002674547?sid=100',
 'https://n.news.naver.com/mnews/article/422/0000641637?sid=100',
 'https://n.news.naver.com/mnews/article/056/0011650330?sid=100',
 'https://n.news.naver.com/mnews/article/008/0004991194?sid=100',
 'https://n.news.naver.com/mnews/article/660/0000054378?sid=100']

In [15]:
# 정치
politics_titles = []
politics_articles = []
politics_press_companies = []
politics_categories= '정치'

current_call = 1
max_page = 6
last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

while current_call <= last_call: # 조건문이 참인 "동안" 실행
    
#     print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
    
    politics_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100' + "&start=" + str(current_call)
    politics_web = requests.get(politics_url).content
    politics_source = BeautifulSoup(politics_web, 'html.parser')

    politics_urls_list = []
    for politics_urls in politics_source.find_all('a', {'class' : "sh_text_headline nclicks(cls_pol.clsart)"}):
        if politics_urls["href"].startswith("https://n.news.naver.com"):
            politics_urls_list.append(politics_urls["href"])

    for politics_url in politics_urls_list:
        try:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            politics_web_news = requests.get(politics_url, headers=headers).content
            politics_source_news = BeautifulSoup(politics_web_news, 'html.parser')

            politics_title = politics_source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
#             print('Processing article : {}'.format(politics_title))

            politics_date = politics_source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

            politics_article = politics_source_news.find('div', {'id' : 'newsct_article'}).get_text()
            politics_article = politics_article.replace("\n", "")
            politics_article = politics_article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            politics_article = politics_article.replace("동영상 뉴스       ", "")
            politics_article = politics_article.replace("동영상 뉴스", "")
            politics_article = politics_article.strip()
            

            politics_press_company = politics_source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
            
            politics_titles.append(politics_title)
            politics_articles.append(politics_article)
            politics_press_companies.append(politics_press_company)
        except:
            pass
            
    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    time.sleep(5)
    current_call += 10

    
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
politics_article_df = pd.DataFrame({'Title':politics_titles, 
                           'Article':politics_articles,
                            'Category': politics_categories,
                           'PressCompany':politics_press_companies})

politics_article_df.to_csv('politics_article_result.csv', index=False, encoding='utf-8')
politics_article_df.head()

,Title,Article,Category,PressCompany
0,"이수정 “배현진, 여자라 피해자 된 걸 수도…우발적 사건”",국민의힘 배현진 의원(서울 송파을)이 25일 오후 서울 강남구 신사동의 한 건물에서...,정치,이데일리
1,中 외교부 부부장 방북…수교 75주년 교류확대 논의할 듯(종합),[서울=뉴시스] 1월25일 중국 외교부 쑨웨이둥 부부장을 단장으로 하는 중국 외교부...,정치,뉴시스
2,"'배현진 테러'에 여야 ""증오의 정치 멈춰야""…그럼에도 정쟁은 진행형","여야, 자성 목소리 이어졌지만 정쟁은 계속전문가 ""정치권 행태가 국민 분노 촉발"" ...",정치,아시아경제
3,"尹, 중대재해처벌법 유예 불발에 ""야당 무책임한 행위 유감""","""중소기업 어려움과 민생경제 도외시""산업 현장 혼란 및 부작용 최소화 지시윤석열 대...",정치,뉴스1
4,"정의당 비례대표 이은주 사직안, 국회 본회의서 가결",정의당 비례대표인 이은주 의원의 국회의원직 사직안이 오늘(25일) 열린 국회 본회의...,정치,KBS


In [16]:
# 경제
economy_titles = []
economy_articles = []
economy_press_companies = []
economy_categories= '경제'

current_call = 1
max_page = 6
last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

while current_call <= last_call: # 조건문이 참인 "동안" 실행
    
#     print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
    
    economy_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101' + "&start=" + str(current_call)
    economy_web = requests.get(economy_url).content
    economy_source = BeautifulSoup(economy_web, 'html.parser')

    economy_urls_list = []
    for economy_urls in economy_source.find_all('a', {'class' : "sh_text_headline nclicks(cls_eco.clsart)"}):
        if economy_urls["href"].startswith("https://n.news.naver.com"):
            economy_urls_list.append(economy_urls["href"])

    for economy_url in economy_urls_list:
        try:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            economy_web_news = requests.get(economy_url, headers=headers).content
            economy_source_news = BeautifulSoup(economy_web_news, 'html.parser')

            economy_title = economy_source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
#             print('article : {}'.format(economy_title))

            economy_date = economy_source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

            economy_article = economy_source_news.find('div', {'id' : 'newsct_article'}).get_text()
            economy_article = economy_article.replace("\n", "")
            economy_article = economy_article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            economy_article = economy_article.replace("동영상 뉴스       ", "")
            economy_article = economy_article.replace("동영상 뉴스", "")
            economy_article = economy_article.strip()

            economy_press_company = economy_source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
            
            economy_titles.append(economy_title)
            economy_articles.append(economy_article)
            economy_press_companies.append(economy_press_company)
        except:
            pass
            
    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    time.sleep(5)
    current_call += 10

    
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
economy_article_df = pd.DataFrame({'Title':economy_titles, 
                           'Article':economy_articles,
                            'Category': economy_categories,
                           'PressCompany':economy_press_companies})
economy_article_df.to_csv('economy_article_result.csv', index=False, encoding='utf-8')
economy_article_df.head()

,Title,Article,Category,PressCompany
0,25년 만에 경제성장률 日에 역전당해… 올해 재역전 예상,한국 지난해 실질 GDP 성장률 1.4%IMF 추산치 日 2.0%에 뒤지면서25년 ...,경제,국민일보
1,SK하이닉스 마침내 흑자 전환…반도체업계 '불황 탈출' 본격화,국내 반도체업계가 장기간 이어진 불황 국면을 벗어나 인공지능(AI)용 고부가가치 제...,경제,SBS
2,GM의 미래로 떠오른 한국사업장...새해부터 총괄부사장 방한,지난 24~25일 이틀간 부평·창원공장 점검제너럴 모터스(GM) 제럴드 존슨 글로벌...,경제,이코노미스트
3,1인당 하루에 밥 '한 공기 반' 먹는다…30년 전의 절반 수준,작년 1인당 하루 평균 쌀 154.6g 소비…전년보다 0.6% 감소서울 시내의 한 ...,경제,뉴스1
4,평당 분양가 1억 '포제스 한강'에 6백여명 몰렸다...경쟁률 6.1대 1,서울 광장동 한강변 아파트 '포제스 한강' 조감도 (출처='포제스 한강' 홈페이지)...,경제,JTBC


In [17]:
# 사회 https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102
society_titles = []
society_articles = []
society_press_companies = []
society_categories= '사회'

current_call = 1
max_page = 6
last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

while current_call <= last_call: # 조건문이 참인 "동안" 실행
    
#     print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
    
    society_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102' + "&start=" + str(current_call)
    society_web = requests.get(society_url).content
    society_source = BeautifulSoup(society_web, 'html.parser')

    society_urls_list = []
    for society_urls in society_source.find_all('a', {'class' : "sh_text_headline nclicks(cls_nav.clsart)"}):
        if society_urls["href"].startswith("https://n.news.naver.com"):
            society_urls_list.append(society_urls["href"])

    for society_url in society_urls_list:
        try:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            society_web_news = requests.get(society_url, headers=headers).content
            society_source_news = BeautifulSoup(society_web_news, 'html.parser')

            society_title = society_source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
#             print('article : {}'.format(society_title))

            society_date = society_source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

            society_article = society_source_news.find('div', {'id' : 'newsct_article'}).get_text()
            society_article = society_article.replace("\n", "")
            society_article = society_article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            society_article = society_article.replace("동영상 뉴스       ", "")
            society_article = society_article.replace("동영상 뉴스", "")
            society_article = society_article.strip()

            society_press_company = society_source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
            
            society_titles.append(society_title)
            society_articles.append(society_article)
            society_press_companies.append(society_press_company)
        except:
            pass
            
    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    time.sleep(5)
    current_call += 10

    
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
society_article_df = pd.DataFrame({'Title':society_titles, 
                           'Article':society_articles,
                            'Category': society_categories,
                           'PressCompany':society_press_companies})

society_article_df.to_csv('society_article_result.csv', index=False, encoding='utf-8')
society_article_df.head()

,Title,Article,Category,PressCompany
0,김천수 중앙회장 고향사랑기부제 '1호' 기부,도지사에 500만원 전달 '솔선수범'▲ 김천수 강원특별자치도민회중앙회 회장이 김진태...,사회,강원도민일보
1,‘46억원 횡령’ 건보 공단 40대 직원 검찰 송치,◇강원경찰청에 압송된 건보 공단 46억원 횡령 피의자 A(46)씨. 사진=연합뉴스국...,사회,강원일보
2,“미용실서 연예인 나오길 기다렸다” 배현진 습격한 중학생 진술… 촉법소년은 아니었다,"피의자, 2009년생 강남 인근 중학생으로 촉법소년에는 해당하지 않아 국민의힘...",사회,세계일보
3,차량 훔치고 달아난 40대 흉기 저항까지...경찰 '실탄' 쏴 제압,다리 부위에 실탄 쏴 현행범 체포사진=게티이미지뱅크차량을 훔쳐 도주하던 한 40대 ...,사회,MBN
4,83세 '최고령 수험생' 할머니 숙명여대 합격,2024학년도 대학수학능력시험의 최고령 수험생이었던 김정자 할머니가 숙명여대에 합격...,사회,연합뉴스TV


In [18]:
# 생활문화 https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103

culture_titles = []
culture_articles = []
culture_press_companies = []
culture_categories= '생활/문화'

current_call = 1
max_page = 6
last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

while current_call <= last_call: # 조건문이 참인 "동안" 실행
    
#     print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
    
    culture_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103' + "&start=" + str(current_call)
    culture_web = requests.get(culture_url).content
    culture_source = BeautifulSoup(culture_web, 'html.parser')

    culture_urls_list = []
    for culture_urls in culture_source.find_all('a', {'class' : "sh_text_headline nclicks(cls_lif.clsart)"}):
        if culture_urls["href"].startswith("https://n.news.naver.com"):
            culture_urls_list.append(culture_urls["href"])

    for culture_url in culture_urls_list:
        try:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            culture_web_news = requests.get(culture_url, headers=headers).content
            culture_source_news = BeautifulSoup(culture_web_news, 'html.parser')

            culture_title = culture_source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
#             print('article : {}'.format(culture_title))

            culture_article = culture_source_news.find('div', {'id' : 'newsct_article'}).get_text()
            culture_article = culture_article.replace("\n", "")
            culture_article = culture_article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            culture_article = culture_article.replace("동영상 뉴스       ", "")
            culture_article = culture_article.replace("동영상 뉴스", "")
            culture_article = culture_article.strip()

            culture_press_company = culture_source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
            
            culture_titles.append(culture_title)
            culture_articles.append(culture_article)
            culture_press_companies.append(culture_press_company)
        except:
            pass
            
    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    time.sleep(5)
    current_call += 10

    
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
culture_article_df = pd.DataFrame({'Title':culture_titles, 
                           'Article':culture_articles,
                            'Category': culture_categories,
                           'PressCompany':culture_press_companies})

culture_article_df.to_csv('culture_article_result.csv', index=False, encoding='utf-8')
culture_article_df.head()

,Title,Article,Category,PressCompany
0,"르노 XM3, 모로코서 '올해의 자동차' 수상","""아름다운 디자인에 뛰어난 성능""XM3 TCe 260 인스파이어. 르노코리아자동차 ...",생활/문화,노컷뉴스
1,"강원2024 경기 보고, 강릉 로컬100도 즐기고",유인촌 문화체육관광부 장관이 26일 2024 강원 동계청소년올림픽이 열리고 있는 강...,생활/문화,서울신문
2,"[WE+] 딱 열흘간 열리는 겨울왕국, 영원히 남을 순백의 추억",오늘부터 내달 4일까지 태백 일원눈조각·얼음 썰매장·공연·체험 등31회 맞아 31개...,생활/문화,강원도민일보
3,"""이건 사야돼""…벤츠 E클래스 한정판, 나오자마자 '완판'",벤츠 E-클래스 핵심 시장 한국신형 출시 기념해 '한정판' 선보여더 뉴 E-클래스 ...,생활/문화,한국경제
4,"정부, 근로자 여행경비 50% 지원 '선착순 모집'",20만원 적립하면 기업·정부가 10만원씩15만명 목표…중소기업·소상공인 등 대상만족...,생활/문화,아시아경제


In [19]:
# IT과학 https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105


science_titles = []
science_articles = []
science_press_companies = []
science_categories= 'IT/과학'

current_call = 1
max_page = 6
last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

while current_call <= last_call: # 조건문이 참인 "동안" 실행
    
#     print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
    
    science_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105' + "&start=" + str(current_call)
    science_web = requests.get(science_url).content
    science_source = BeautifulSoup(science_web, 'html.parser')

    science_urls_list = []
    for science_urls in science_source.find_all('a', {'class' : "sh_text_headline nclicks(cls_sci.clsart)"}):
        if science_urls["href"].startswith("https://n.news.naver.com"):
            science_urls_list.append(science_urls["href"])

    for science_url in science_urls_list:
        try:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            science_web_news = requests.get(science_url, headers=headers).content
            science_source_news = BeautifulSoup(science_web_news, 'html.parser')

            science_title = science_source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
#             print('article : {}'.format(science_title))

            science_article = science_source_news.find('div', {'id' : 'newsct_article'}).get_text()
            science_article = science_article.replace("\n", "")
            science_article = science_article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            science_article = science_article.replace("동영상 뉴스       ", "")
            science_article = science_article.replace("동영상 뉴스", "")
            science_article = science_article.strip()

            science_press_company = science_source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
            
            science_titles.append(science_title)
            science_articles.append(science_article)
            science_press_companies.append(science_press_company)
        except:
            pass
            
    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    time.sleep(5)
    current_call += 10

    
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
science_article_df = pd.DataFrame({'Title':science_titles, 
                           'Article':science_articles,
                            'Category': science_categories,
                           'PressCompany':science_press_companies})

science_article_df.to_csv('science_article_result.csv', index=False, encoding='utf-8')
science_article_df.head()

,Title,Article,Category,PressCompany
0,제4통신사 선정 위한 주파수 경매 돌입…세종텔레콤 포기,"제4 통신사 선정을 위한 28기가 헤르츠 대역 경매 첫 날, 입찰 참여 업체 세 곳...",IT/과학,KBS
1,갤럭시 S24 시리즈 사전 개통 개시 [뉴시스Pic],[서울=뉴시스] KT가 갤럭시S24 시리즈 사전예약에서 온라인 한정으로 판매한 1T...,IT/과학,뉴시스
2,"갤S24, 지원금 받아도 최소 100만원 넘어…선택약정할인이 유리","이동통신 3사, 갤럭시S24 개통 시작이동통신사들이 26일 갤럭시S24 시리즈 사전...",IT/과학,한겨레
3,미 “달에 위성 보내주겠다” 제안했지만…정작 한국은 예산 없어 ‘무산’,"NASA, 지난해 10월 한국 등에 제안정부, 국회에 예산 요청에 반영 안 돼202...",IT/과학,경향신문
4,"1000km 달리는 꿈의 전기차, 젤(gel)로 가능하다","포스텍 박수진 교수팀, 전자빔 활용 일체형 실리콘·젤 전해질 시스템 개발젤(gel)...",IT/과학,세계일보


In [21]:
# 세계 https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=104

world_titles = []
world_articles = []
world_press_companies = []
world_categories= '세계'

current_call = 1
max_page = 6
last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

while current_call <= last_call: # 조건문이 참인 "동안" 실행
    
#     print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
    
    world_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=104' + "&start=" + str(current_call)
    world_web = requests.get(world_url).content
    world_source = BeautifulSoup(world_web, 'html.parser')

    world_urls_list = []
    for world_urls in world_source.find_all('a', {'class' : "sh_text_headline nclicks(cls_wor.clsart)"}):
        if world_urls["href"].startswith("https://n.news.naver.com"):
            world_urls_list.append(world_urls["href"])

    for world_url in world_urls_list:
        try:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            world_web_news = requests.get(world_url, headers=headers).content
            world_source_news = BeautifulSoup(world_web_news, 'html.parser')

            world_title = world_source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
#             print('article : {}'.format(science_title))

            world_article = world_source_news.find('div', {'id' : 'newsct_article'}).get_text()
            world_article = world_article.replace("\n", "")
            world_article = world_article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            world_article = world_article.replace("동영상 뉴스       ", "")
            world_article = world_article.replace("동영상 뉴스", "")
            world_article = world_article.strip()

            world_press_company = world_source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
            
            world_titles.append(world_title)
            world_articles.append(world_article)
            world_press_companies.append(world_press_company)
        except:
            pass
            
    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    time.sleep(5)
    current_call += 10

    
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
world_article_df = pd.DataFrame({'Title':world_titles, 
                           'Article':world_articles,
                            'Category': world_categories,
                           'PressCompany':world_press_companies})

world_article_df.to_csv('world_article_result.csv', index=False, encoding='utf-8')
world_article_df.head()

,Title,Article,Category,PressCompany
0,"외신, 이재명·배현진 잇단 피습에 ""韓 정치 분노 강해져""","WP·AP·NYT·닛케이 등 보도…""극도로 양극화된 정치상황 우려""배현진 국민의힘 ...",세계,시사저널
1,물밑에서 이렇게까지…삼성 AI폰 출시에 애플 ‘발등의 불’,중국 베이징 애플 매장에서 방문객들이 애플 제품을 구경하고 있다. [AP][헤럴드경...,세계,헤럴드경제
2,"""트럼프, 바이든에 6%p 앞서""…""67% 재대결 부정적""",【 앵커멘트 】 오는 11월 미국 대선에서 전현직 대통령의 재대결 가능성이 유력해...,세계,MBN
3,2024년 증시도 AI… MS·구글·SAP 역대 최고가,[서울경제] 인공지능(AI) 확산 기대감에 마이크로소프트(MS) 시가총액이 장중 3...,세계,서울경제
4,"줄 잇는 외신 보도 ""김건희 가방 논란, 국힘 혼란 빠뜨려""","가방 수수 의혹 영국, 홍콩 등 외신 연이어 보도…<타임> ""영부인, 이미 과거 논...",세계,프레시안
